# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, LSTM, LSTMCell, \
  SimpleRNNCell
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K
import functools
def pipe(*fns):
  return functools.reduce(lambda f, g: lambda x: f(g(x)), reversed(fns))


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')  # shape: 60000, 28, 28
x_test = x_test.astype('float32')  # shape: 10000, 28, 28
x_train /= 255  # normalizacja wartości do przedziału [0, 1]
x_test /= 255

y_train = to_categorical(y_train, 10)  # zamiana etykiety na one-hot encoding; np. 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_test = to_categorical(y_test, 10)

## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://www.tensorflow.org/guide/keras/rnn

https://www.tensorflow.org/guide/function

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [4]:
class RecurrentModel(Model):

  def __init__(self, num_classes=10):
    super(RecurrentModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # Define your layers here.
    self.lstm_1 = LSTM(128, activation='relu')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    return pipe(
      self.lstm_1,
      self.dense_1,
    )(inputs)

model = RecurrentModel(num_classes=10)

In [ ]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=4)

Epoch 1/4
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1875/1875 [==============================] - 20s 10ms/step - loss: 0.5387 - accuracy: 0.8305
Epoch 2/4
 889/1875 [=============>................] - ETA: 9s - loss: 0.1542 - accuracy: 0.9548

### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu, natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w Kerasie zwracają tylko wyniki przetwarzania ostatniego
kroku (otrzymują tensor 3-wymiarowy, zwracają tensor 2-wymiarowy). Jeśli chcesz zwrócić sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej, musisz ustawić parametr return_sequences=True.


In [ ]:
class RecurrentBeans(Model):
  def __init__(self, num_classes=10):
    super(RecurrentBeans, self).__init__(name='RecurrentBeans')
    self.num_classes = num_classes
    # Define your layers here.
    self.lstm_1 = LSTM(128, activation='relu', return_sequences=True)
    self.lstm_2 = LSTM(128, activation='relu')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    return pipe(
      self.lstm_1,
      self.lstm_2,
      self.dense_1,
    )(inputs)

model = RecurrentBeans(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=4)

### Zadanie 2 
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu SimpleRNNCell.

Cell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki SimpleRNNCell (liczba kroków to liczba elementów w sekwencji). Aby wywołać SimpleRNNCell dla pojedynczego wejścia i stanu należy użyć jej metody ```call``` analogicznie jak w przypadku własnych modeli (tzn. ```my_model(input)```). 



Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i **listy macierzy** (w dokumentacji jest błąd, że ma to być macierz) stanów ukrytych poprzedniego kroku (SimpleRNNCell ma jeden stan, LSTMCell w następnym zadaniu ma dwa stany).

Trzeba zainicjalizować ukryty stan warstwy wartościami początkowymi (można wykorzystać rozkład normalny - tf.random.normal).

In [ ]:
class RNNRecurrentBeans(Model):
  def __init__(self, num_classes):
    super().__init__(name='RNNRecurrentBeans')
    self.num_classes = num_classes
    self.cell = SimpleRNNCell(128, activation='relu')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    h = tf.random.normal([inputs.shape[0], self.cell.units])

    for n in range(inputs.shape[1]):
      x, h = self.cell(inputs[:, n, :], h)

    return self.dense_1(x)

model = RNNRecurrentBeans(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=4)

### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell (LSTMCell ma dwa stany ukryte).

In [ ]:
class LSTMRecurrentBeans(Model):
  def __init__(self, num_classes):
    super().__init__(name='LSTMRecurrentBeans')
    self.num_classes = num_classes
    self.cell = LSTMCell(128, activation='relu')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def call(self, inputs):
    shape = (inputs.shape[0], self.cell.units)
    h = [tf.random.normal(shape), tf.random.normal(shape)]

    for n in range(inputs.shape[1]):
      x, h = self.cell(inputs[:, n, :], h)

    return self.dense_1(x)

model = LSTMRecurrentBeans(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=4)

### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wywołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [ ]:
class CoolRecurrentBeans(Model):
  def __init__(self, num_classes):
    super().__init__(name='CoolRecurrentBeans')
    self.num_classes = num_classes
    self._cell = Dense(128, activation='relu')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def cell(self, x, h):
    return self._cell(K.concatenate([x, h]))

  def call(self, inputs):
    h = tf.random.normal((inputs.shape[0], self._cell.units))

    for n in range(inputs.shape[1]):
      h = self.cell(inputs[:, n, :], h)

    return self.dense_1(h)

model = CoolRecurrentBeans(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=4)

### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania warstwy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [ ]:
class LSTMCoolRecurrentBeans(Model):
  def __init__(self, num_classes):
    super().__init__(name='Model5')
    self.num_classes = num_classes
    self.units = 128
    self.dense_i = Dense(self.units, activation='sigmoid')
    self.dense_f = Dense(self.units, activation='sigmoid')
    self.dense_o = Dense(self.units, activation='sigmoid')
    self.dense_c = Dense(self.units, activation='tanh')
    self.dense_1 = Dense(num_classes, activation='softmax')

  def cell(self, x, h):
    c_last, h_last = h
    hx = K.concatenate([h_last, x])
    c_next = self.dense_f(hx) * c_last + self.dense_i(hx) * self.dense_c(hx)
    h_next = self.dense_o(hx) * K.tanh(c_next)
    return h_next, (c_next, h_next)

  def call(self, inputs):
    shape = (inputs.shape[0], self.units)
    h = [tf.random.normal(shape), tf.random.normal(shape)]

    for n in range(inputs.shape[1]):
      x, h = self.cell(inputs[:, n, :], h)

    return self.dense_1(x)

model = LSTMCoolRecurrentBeans(num_classes=10)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=2)